<h1> Setup



In [2]:
from dotenv import load_dotenv

load_dotenv()


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

<h1> Example Email

In [10]:
email_conversation = """From: IMF (xxx@imf.org)
To: 신우일(wishks14@gmail)
Subject: IMF TEST

Dear Candidate,

 

Please find attached the test for the HQTA Financial Sector Expert/Senior Financial Sector Expert (Central Bank Governance Expert) position, 
with your responses due by noon (Washington, D.C. time), Monday, October 14, 2024. Kindly respond directly to this email with your test responses, without changing the subject line, emailing MCMCOCOOR@imf.org.
In your test responses, please do not write down your name anywhere in the document, and kindly send it back as a Word file rather than a PDF.

Kindly note that given the large number of applicants, we will be unable to provide flexibility in the dates for the test and the hiring unit will not be able to respond to individual inquiries. Only successful candidates will be contacted.

Best regards,
 
JJ
Administrative Coordinator
Central Bank Operations (CO) 
INTERNATIONAL MONETARY FUND
700 19th Street, NW  I  HQ1-06-569.1 I  Washington, D.C. 20431
"""

<h1> Define Pydantic style Class

In [ ]:
# The PydanticOutputParser with the EmailSummary model will output a set of format instructions similar to this:
'''
{
    "person": "string", // 메일을 보낸 사람
    "email": "string", // 메일을 보낸 사람의 이메일 주소
    "subject": "string", // 메일 제목
    "summary": "string", // 메일 본문을 요약한 텍스트
    "date": "string" // 메일 본문에 언급된 미팅 날짜와 시간
}
'''


class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# Print
print(parser.get_format_instructions())

<h1> Prompt

In [ ]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

<h1> Generate chain

In [ ]:
chain = prompt | llm
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }

In [ ]:
# PydanticOutputParser 를 사용하여 결과를 파싱
structured_output = parser.parse(response)
print(structured_output)